# Task 3 分类数据的检验

> 分类型数据也称为频数数据。在数据样本中，我们称落入某一个特定分组的样本数量为频数；当分组的维度只有1时，我们称这样的数据为单因素频数表；当分组的维度为2时，我们称这样的数据为列联表数据。

<img src='pic/Task3_1.png'>

**卡方检验的适用范围**    
> 卡方检验适用于单因素频数表、双因素频数表中的与列联表。    
> **单因素频数表**
> - 在单因素频数表中，每一类的绝对频数不能小于5。
> **$2 × 2$列联表**
> - 样本总量$\sum o_i > 40$，且所有期望频数$e_i > 5$，可使用Pearson卡方检验
> - 样本总量$\sum o_i > 40$，但存在期望频数$1 < e_i < 5$，可使用连续型校正的卡方检验
> - 若样本总量$\sum o_i < 40$，或存在期望频数$1 < e_i$，建议使用Fisher检验
> **$r × c$列联表**
> - 表中期望频数$e_i < 5$的格子不能超过1/5。
> - 不得出现期望频数$1 < e_i$的情况。

**Fisher检验的适用范围**    
> Fisher检验仅仅适用于双因素频数表中的列联表。    
> Fisher检验在列联表中适用范围内很广，弥补了卡方检验的缺点。

## 作业

一个淘宝网购商家搜集了一年中每天的订单数$X$，除去春节期间及双十一前后外，按330天记，数据如下

<img src='pic/淘宝.png'>

请用卡方分布验证订单数是否泊松分布。已知：通过极大似然估计得知泊松分布参数$\lambda = 5.3$

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = {'counts': list(range(14))+[16], 
        'observe':[3,6,21,46,48,61,52,42,27,11,6,4,1,1,1]}
df = pd.DataFrame(data)
df

,counts,observe
0,0,3
1,1,6
2,2,21
3,3,46
4,4,48
5,5,61
6,6,52
7,7,42
8,8,27
9,9,11


将实际频数小于5的类别合并

In [3]:
df.loc[11,'observe'] = df.iloc[11:,:]['observe'].sum()
df = df[:-3]
df.loc[1,'observe'] = df.loc[0,'observe'] + df.loc[1,'observe']
df = df.drop(df.loc[0])
df

,counts,observe
1,1,9
2,2,21
4,4,48
5,5,61
6,6,52
7,7,42
8,8,27
9,9,11
10,10,6
11,11,7


In [4]:
Poiss=stats.poisson(mu=5.3)
df['prop']=Poiss.pmf(df['counts'])

针对前面小于5的列进行修正

In [5]:
df.loc[1, 'prop']= Poiss.cdf(1)
df.loc[11, 'prop']= 1-Poiss.cdf(10)

用理论频率乘样本数，得到理论频数

In [6]:
df['T_counts']=330*df['prop']
df

,counts,observe,prop,T_counts
1,1,9,0.031447,10.377524
2,2,21,0.070107,23.135289
4,4,48,0.164109,54.155856
5,5,61,0.173955,57.405207
6,6,52,0.153660,50.707933
7,7,42,0.116343,38.393149
8,8,27,0.077077,25.435461
9,9,11,0.045390,14.978661
10,10,6,0.024057,7.938690
11,11,7,0.020000,6.599886


使用卡方检验，比较实际频数与理论频数的差别

In [7]:
chi=stats.chisquare(df['observe'], df['T_counts'], ddof=1)
chi

Power_divergenceResult(statistic=3.3272976229781475, pvalue=0.9121721312635175)

$p=0.912$，不能拒绝原假设，因此我们可以认为样本的总体，即订单数服从泊松分布。